# Combine Data (not from prop)

Data has now been filtered for the parcel numbers in the existing dataset. The end goal is to have a dataset with one row for each parcel in which to build the model. **Imputation will occur in a separate notebook!**

However, we can't forget to explore the existing data for trends or insight along the way, as well as engineer features as we see fit.

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pandas as pd
import numpy as np

tci = pd.read_csv('model_data/tci_with_prop.csv')
ppns = set(tci['parcel'])
ppns_num = set(tci['parcel'].apply(lambda x: x.replace('-','')))

## Arms length sales

In [3]:
al = pd.read_csv('clean_data/armslength_tci.csv')
al = al.drop_duplicates()

In [4]:
al.shape

(5497, 86)

In [6]:
al.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5497 entries, 0 to 5496
Data columns (total 86 columns):
AMOUNT_PAID                     5497 non-null float64
ASSUMED_LOAN_AMOUNT             5497 non-null int64
AUTO_FILE_NUMBER                3696 non-null object
BUILDINGS_ON_LAND               5497 non-null int64
CAMA_INV_FILE                   5497 non-null object
CAMA_SALE_FILE                  5497 non-null object
CHANGE_TIMESTAMP                5077 non-null object
CONVEYANCE_FEE                  5497 non-null float64
DEED_TYPE                       5475 non-null object
EXEMPTCODE                      5497 non-null int64
EXEMPTCODE_DESCR                1 non-null object
FEE_CHECK_NUM                   1067 non-null object
FEE_PAID_BY                     4011 non-null object
GIFT                            5497 non-null int64
GRANTEE1                        5497 non-null object
GRANTOR1                        5497 non-null object
GRANTOR_IS_MORTGAGEE            5497 non-null int6

In [36]:
al[['GRANTOR_IS_MORTGAGEE','PROPERTY_NUMBER']].groupby

,GRANTOR_IS_MORTGAGEE,PROPERTY_NUMBER
0,0,129-26-066
1,0,129-16-081
2,0,129-23-164
3,0,128-13-046
4,0,129-26-039
5,0,128-24-082
6,0,139-13-001
7,0,129-19-081
8,0,137-04-049
9,0,137-05-019


In [45]:
tem = pd.merge(tci, al[['GRANTOR_IS_MORTGAGEE','PROPERTY_NUMBER']].groupby('PROPERTY_NUMBER').sum(), \
               how='left', left_on='parcel', right_index=True)

In [50]:
tem.loc[tem['GRANTOR_IS_MORTGAGEE'].isnull(),'GRANTOR_IS_MORTGAGEE'] = 0

In [55]:
t = tem[['vacant','GRANTOR_IS_MORTGAGEE']].groupby('GRANTOR_IS_MORTGAGEE').agg([len,sum])
t.vacant['sum']/t.vacant['len']

GRANTOR_IS_MORTGAGEE
0                       0.114733
1                       0.294118
dtype: float64

In [66]:
sum(al.ASSUMED_LOAN_AMOUNT)

339921

ASSUMED_LOAN_AMOUNT not work investigating -- only 7 properties with entries and non became vacant.

### Number of records for each parcel

In [7]:
tci = pd.merge(tci, al[['PROPERTY_NUMBER','YEAR_OF_SALE']].groupby('PROPERTY_NUMBER').count(), \
               how='left', left_on='parcel', right_index=True)
tci.shape

(13500, 19)

In [8]:
tci.loc[tci.YEAR_OF_SALE.isnull(),'YEAR_OF_SALE'] = 0
tci.rename(columns={'YEAR_OF_SALE':'num_of_sales'}, inplace=True)

In [9]:
t = tci[['vacant','num_of_sales']].groupby('num_of_sales').agg([len,sum])
t.vacant['sum']/t.vacant['len']

num_of_sales
0               0.092249
1               0.158765
2               0.196778
3               0.221053
4               0.257426
5               0.379310
6               0.428571
7               1.000000
dtype: float64

In [10]:
t

vacant     
                len  sum
num_of_sales            
0              9908  914
1              2299  365
2               869  171
3               285   63
4               101   26
5                29   11
6                 7    3
7                 2    2

### Flags for Deeds

In [17]:
def add_deed_fields(field_names):
    for field_name in field_names:
        deed_ppns = set(al[al.DEED_TYPE==field_name].PROPERTY_NUMBER)                
        def get_deed_field(x):
            if x in deed_ppns:
                return 1
            else:
                return 0

        tci[field_name.lower().replace(' ','_')] = tci['parcel'].apply(get_deed_field)
    return tci

In [18]:
tci = add_deed_fields(['QUIT CLAIM DEED', 'WARRANTY DEED', 'LIMITED WARRANTY', 'SURVIVORSHIP DEED','FIDUCIARY DEED'])

In [23]:
tci[['fiduciary_deed','vacant']].groupby('fiduciary_deed').agg([sum,len])

vacant       
                  sum    len
fiduciary_deed              
0                1539  13376
1                  16    124

### STD Name -- flag if has name appears more than 3 times

In [11]:
names = al[['stdname','PROPERTY_NUMBER']].groupby('stdname').count()

In [12]:
multiple_names = set(names[names['PROPERTY_NUMBER']>3].sort('PROPERTY_NUMBER',ascending=False).index)

In [13]:
def get_stdname(x):
    if x['stdname'] in multiple_names:
        return 1
    else:
        return 0

# tci['multiple_names'] = tci['parcel'].apply(get_stdname)

In [14]:
al['mult_name_flag'] = al[['stdname','PROPERTY_NUMBER']].apply(get_stdname, axis=1)

In [15]:
tci = pd.merge(tci, al[['mult_name_flag','PROPERTY_NUMBER']].groupby('PROPERTY_NUMBER').last(), \
               how='left', left_on='parcel', right_index=True)

In [16]:
t = tci[['mult_name_flag','vacant']].groupby('mult_name_flag').agg([sum,len])
print t
print t.vacant['sum']/t.vacant['len']

               vacant      
                  sum   len
mult_name_flag             
0                 544  3076
1                  97   516
mult_name_flag
0                 0.176853
1                 0.187984
dtype: float64


### Sale Valid {0, 1, 2}

In [17]:
tci = pd.merge(tci, al[['SALE_VALID','PROPERTY_NUMBER']].groupby('PROPERTY_NUMBER').last(), \
               how='left', left_on='parcel', right_index=True)

In [18]:
t = tci[['SALE_VALID','vacant']].groupby('SALE_VALID').agg([sum,len])
print t
print t.vacant['sum']/t.vacant['len']

           vacant      
              sum   len
SALE_VALID             
0             385  1937
1               6    29
2             250  1626
SALE_VALID
0             0.198761
1             0.206897
2             0.153752
dtype: float64


## Transfers

In [2]:
t = pd.read_csv('clean_data/transfers_tci.csv')

In [3]:
t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24943 entries, 0 to 24942
Data columns (total 60 columns):
TRANSFER_TO_ID                  24943 non-null int64
WHS_ID                          24943 non-null int64
TRANSFER_HISTORY_ID             24943 non-null int64
TRANSFER_ORDER                  24943 non-null int64
PARCEL_ID                       24943 non-null int64
PROPERTY_NUMBER                 24943 non-null object
GRANTEE1                        24943 non-null object
GRANTOR1                        24910 non-null object
mdate                           24943 non-null object
SALES_AMOUNT                    24943 non-null int64
DEED_TYPE                       24943 non-null object
instrument_number               20025 non-null float64
RECORDED_BOOK                   4939 non-null float64
RECORDED_PAGE                   4939 non-null object
INSTRUMENT_TYPE                 24943 non-null int64
SALE_VALID                      24943 non-null int64
multiproperty_sale              249

In [142]:
t[t.ASSUMED_LOAN_AMOUNT>0].shape

(12, 60)

In [80]:
t.columns

Index([u'TRANSFER_TO_ID', u'WHS_ID', u'TRANSFER_HISTORY_ID', u'TRANSFER_ORDER', u'PARCEL_ID', u'PROPERTY_NUMBER', u'GRANTEE1', u'GRANTOR1', u'mdate', u'SALES_AMOUNT', u'DEED_TYPE', u'instrument_number', u'RECORDED_BOOK', u'RECORDED_PAGE', u'INSTRUMENT_TYPE', u'SALE_VALID', u'multiproperty_sale', u'NUMBER_OF_PROPERTIES_IN_SALE', u'conveyance_number', u'CONVEYANCE_FEE', u'USER_ID', u'RECEIPT_NUMBER', u'AUTO_FILE_NUMBER', u'CHANGE_TIMESTAMP', u'TRANSFER_NUMBER', u'TRANSFER_STATUS', u'MONTH_OF_SALE', u'YEAR_OF_SALE', u'CAMA_SALE_FILE', u'CAMA_INV_FILE', u'TRANSFER_TYPE', u'INSTRUMENT_DATE', u'RECORDED_DATE', u'TRANSFER_FEE', u'PERMISSIVE_FEE', u'AMOUNT_PAID', u'FEE_PAID_BY', u'FEE_CHECK_NUM', u'SALES_SOURCE', u'deed_number', u'EXEMPTCODE', u'ASSUMED_LOAN_AMOUNT', u'PERSONAL_PROPERTY_AMOUNT', u'BUILDINGS_ON_LAND', u'GIFT', u'LAND_CONTRACT', u'LEASE_HOLD', u'LEASED_FEE', u'LIFE_ESTATE', u'MINERAL_RIGHTS_RESERVED', u'MOTHER', u'PARTINTER_EST_TRANSFERRED', u'GRANTOR_IS_MORTGAGEE', u'GRANTOR_IS

In [84]:
t[['PROPERTY_NUMBER','DEED_TYPE']].groupby('DEED_TYPE').count().sort(columns='PROPERTY_NUMBER', ascending=False)

,PROPERTY_NUMBER
DEED_TYPE,
Warranty Deed,6365
Sheriffs Deed,3411
Quit Claim Deed Ex,2975
Quit Claim Deed,2838
Probate Court,1145
Sheriffs Deed Ex,1061
Affidavit,965
Limited Warranty,944
Warranty Deed Ex,712


In [13]:
deeds = ['Warranty Deed','Sheriffs Deed',' Q']'Transfer on Death'

In [14]:
a = pd.merge(tci, t.loc[t.DEED_TYPE==deed, ['DEED_TYPE', 'PROPERTY_NUMBER']].groupby('PROPERTY_NUMBER').count(), \
         how='left', left_on='parcel',right_index=True)

In [15]:
sum(a[a.DEED_TYPE.notnull()].vacant), len(a[a.DEED_TYPE.notnull()].vacant),sum(a[a.DEED_TYPE.notnull()].vacant)*1.0/len(a[a.DEED_TYPE.notnull()].vacant)

(21, 161, 0.13043478260869565)

## County Land Bank

In [23]:
lb = pd.read_csv('clean_data/count_land_bank_tci.csv')

In [24]:
lb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 385 entries, 0 to 384
Data columns (total 16 columns):
parcel            385 non-null object
disp_status       385 non-null object
p_source          385 non-null object
acq_dt            385 non-null object
disp_dt           332 non-null object
assessment        385 non-null object
demo_status       325 non-null object
rehab_status      55 non-null object
public_status     385 non-null object
s_date            305 non-null object
cclrc_dba_date    187 non-null object
cclrc_dc_date     305 non-null object
ab_proceed_dt     241 non-null object
proceed_date      277 non-null object
cclrc_dcp_date    303 non-null object
out_type          336 non-null object
dtypes: object(16)
memory usage: 51.1+ KB


In [25]:
lb.head()

,parcel,disp_status,p_source,acq_dt,disp_dt,assessment,demo_status,rehab_status,public_status,s_date,cclrc_dba_date,cclrc_dc_date,ab_proceed_dt,proceed_date,cclrc_dcp_date,out_type
0,121-32-003,Disposed,Tax Foreclosure,2012-05-21 00:00:00,2013-01-08 00:00:00,Demolition,Demo - Complete,NaN,Disposition,06/01/2012,NaN,08/16/2012,07/07/2012,08/02/2012,08/10/2012,Municipality
1,121-32-038,Disposed,Tax Foreclosure,2012-08-28 00:00:00,2013-08-08 00:00:00,Demolition,Demo - Complete,NaN,Disposition,10/12/2012,11/07/2012,11/16/2012,11/29/2012,12/10/2012,04/05/2013,Municipality
2,121-32-048,Disposed,Tax Foreclosure,2013-02-11 00:00:00,2015-03-10 00:00:00,Demolition,Demo - Complete,NaN,Disposition,05/14/2013,10/28/2013,11/04/2013,11/01/2013,11/18/2013,12/09/2013,Municipality
3,121-32-049,Disposed,Tax Foreclosure,2012-05-30 00:00:00,2013-01-31 00:00:00,Demolition,Demo - Complete,NaN,Disposition,06/15/2012,NaN,09/05/2012,10/12/2012,10/24/2012,12/06/2012,Municipality
4,121-32-050,Disposed,Tax Foreclosure,2012-12-07 00:00:00,2013-01-31 00:00:00,Vacant lot,NaN,NaN,Disposition,NaN,NaN,NaN,NaN,NaN,NaN,Municipality


In [26]:
lb[['assessment','parcel']].groupby('assessment').count()

,parcel
assessment,
Demolition,317
Pass Through,6
Renovate,35
Vacant lot,27


In [27]:
lb_ppns = set(lb.parcel)

In [28]:
def get_lb(x):
    if x in lb_ppns:
        return 1
    else:
        return 0

tci['county_land_bank'] = tci['parcel'].apply(get_lb)

In [29]:
sum(tci.county_land_bank)

385

In [30]:
tci[['county_land_bank','vacant']].groupby('county_land_bank').agg([len, sum])

vacant      
                    len   sum
county_land_bank             
0                 13115  1503
1                   385    52

## Postal Vacancy

In [31]:
pv = pd.read_csv('clean_data/postal_vacancy_tci.csv', parse_dates=3)

In [32]:
pv.head()

,Unnamed: 0,PARCEL,vindall,date
0,10537,121-31-003,Y,2013-02-01 00:00:00
1,10540,121-32-003,Y,2013-02-01 00:00:00
2,10542,121-32-008,Y,2013-02-01 00:00:00
3,10543,121-32-012,Y,2013-02-01 00:00:00
4,10544,121-32-028,Y,2013-02-01 00:00:00


In [33]:
pv = pv.sort('date',ascending=False)

In [34]:
# tci = pd.merge(tci, al[['SALE_VALID','PROPERTY_NUMBER']].groupby('PROPERTY_NUMBER').last(), \
#                how='left', left_on='parcel', right_index=True)

vindall = pv[['PARCEL','vindall']].groupby('PARCEL').first()['vindall']

In [35]:
v_Y = set(vindall[vindall=='Y'].index)

def get_vindall_Y(x):
    if x in v_Y:
        return 1
    else:
        return 0

tci['vindall_Y'] = tci['parcel'].apply(get_vindall_Y)

In [36]:
v_P = set(vindall[vindall=='P'].index)

def get_vindall_P(x):
    if x in v_P:
        return 1
    else:
        return 0

tci['vindall_P'] = tci['parcel'].apply(get_vindall_P)

In [37]:
tci.shape

(13500, 29)

In [38]:
tci[['vindall_Y','vacant']].groupby('vindall_Y').agg([sum, len])

vacant       
             sum    len
vindall_Y              
0            977  12049
1            578   1451

In [39]:
tci[['vindall_P','vacant']].groupby('vindall_P').agg([sum, len])

vacant       
             sum    len
vindall_P              
0           1550  13448
1              5     52

In [40]:
len(vindall)

1503

In [41]:
tci.to_csv('model_data/tci_2_1.csv', index=False)